####  Run this cell to set up and start your interactive session.


In [5]:
%session_id_prefix native-iceberg-sql-
%glue_version 4.0
%idle_timeout 60
%%configure 
{
  "--conf": "spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
  "--datalake-formats": "iceberg",
  "--enable-continuous-cloudwatch-log": "true",
  "--enable-continuous-log-filter": "false"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Setting session ID prefix to native-iceberg-sql-
Setting Glue version to: 4.0
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--conf': 'spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions', '--datalake-formats': 'iceberg', '--enable-continuous-cloudwatch-log': 'true', '--enable-continuous-log-filter': 'false'}


In [1]:

catalog_name = "glue_catalog"
bucket_name = "<bucket-name>"
bucket_prefix = ""
database_name = "iceberg_dataframe"
table_name = "product"
warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"

Trying to create a Glue session for the kernel.
Session Type: glueetl
Idle Timeout: 60
Session ID: 93510a54-356d-425b-a25d-2a0509ba6aa5
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
--datalake-formats iceberg
--enable-continuous-cloudwatch-log true
--enable-continuous-log-filter false
Waiting for session 93510a54-356d-425b-a25d-2a0509ba6aa5 to get into ready status...
Session 93510a54-356d-425b-a25d-2a0509ba6aa5 has been created.



In [3]:
import sys
import json 
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime

spark = SparkSession.builder \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", f"{warehouse_path}") \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()
     

In [4]:
query = f"""
DROP TABLE IF EXISTS {catalog_name}.{database_name}.{table_name}
"""
spark.sql(query)

DataFrame[]


In [5]:
query = f"""
CREATE DATABASE IF NOT EXISTS {catalog_name}.{database_name}
"""
spark.sql(query)


DataFrame[]


In [6]:
# Define the schema of the raw customer table
customer_schema = StructType(
            [ StructField("c_customer_sk",IntegerType(),True),
              StructField("c_customer_id",StringType(),True),
              StructField("c_current_cdemo_sk",IntegerType(),True),
              StructField("c_current_hdemo_sk",IntegerType(),True),
              StructField("c_current_addr_sk",IntegerType(),True),
              StructField("c_first_shipto_date_sk",IntegerType(),True),
              StructField("c_first_sales_date_sk",IntegerType(),True),
              StructField("c_salutation",StringType(),True),
              StructField("c_first_name",StringType(),True),
              StructField("c_last_name",StringType(),True),
              StructField("c_preferred_cust_flag",StringType(),True),
              StructField("c_birth_day",IntegerType(),True),
              StructField("c_birth_month",IntegerType(),True),
              StructField("c_birth_year",IntegerType(),True),
              StructField("c_birth_country", StringType(), True),
              StructField("c_login", StringType(), True),
              StructField("c_email_address", StringType(), True),
              StructField("c_last_review_date_sk", IntegerType(), True)
              ]
              )


In [7]:
df_customer = spark.read.schema(customer_schema).\
                format("csv").options(header=True,delimiter="|").\
                load("s3://redshift-downloads/TPC-DS/2.13/100GB/customer/")
print(f'df_customer - rows: {df_customer.count()}, columns: {len(df_customer.columns)}')

df_customer - rows: 1999999, columns: 18


In [11]:
f"{warehouse_path}/{database_name}/{table_name}"

's3://iceberg-poc-primarybuc-061588638285//iceberg_dataframe/product'


In [12]:
df_customer.writeTo(f"{catalog_name}.{database_name}.{table_name}") \
    .tableProperty("format-version", "2") \
    .tableProperty("location", f"{warehouse_path}{database_name}/{table_name}") \
    .createOrReplace()
     

In [14]:
spark.sql(f"""
SELECT * FROM {catalog_name}.{database_name}.{table_name} LIMIT 5
""").show()

+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+---------------+-------+--------------------+---------------------+
|c_customer_sk|   c_customer_id|c_current_cdemo_sk|c_current_hdemo_sk|c_current_addr_sk|c_first_shipto_date_sk|c_first_sales_date_sk|c_salutation|c_first_name|c_last_name|c_preferred_cust_flag|c_birth_day|c_birth_month|c_birth_year|c_birth_country|c_login|     c_email_address|c_last_review_date_sk|
+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+---------------+-------+--------------------+---------------------+
|            2|AAAAAAAACAAAAAAA|            819667|              1461|           681655|            

In [15]:


spark.table(f"{catalog_name}.{database_name}.{table_name}") \
    .show()
     



+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+------------------+-------+--------------------+---------------------+
|c_customer_sk|   c_customer_id|c_current_cdemo_sk|c_current_hdemo_sk|c_current_addr_sk|c_first_shipto_date_sk|c_first_sales_date_sk|c_salutation|c_first_name|c_last_name|c_preferred_cust_flag|c_birth_day|c_birth_month|c_birth_year|   c_birth_country|c_login|     c_email_address|c_last_review_date_sk|
+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+------------------+-------+--------------------+---------------------+
|            2|AAAAAAAACAAAAAAA|            819667|              1461|           681655|   

In [16]:

spark.table(f"{catalog_name}.{database_name}.{table_name}.history") \
    .show()
     
    
    

+--------------------+-------------------+---------+-------------------+
|     made_current_at|        snapshot_id|parent_id|is_current_ancestor|
+--------------------+-------------------+---------+-------------------+
|2024-09-19 14:38:...|1437657406398650969|     null|               true|
+--------------------+-------------------+---------+-------------------+


In [17]:

df_customer_100B = spark.read.schema(customer_schema).\
                format("csv").options(header=True,delimiter="|").\
                load("s3://redshift-downloads/TPC-DS/1TB/customer/")

In [18]:
df_customer_100B.show()

+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+------------------+-------+--------------------+---------------------+
|c_customer_sk|   c_customer_id|c_current_cdemo_sk|c_current_hdemo_sk|c_current_addr_sk|c_first_shipto_date_sk|c_first_sales_date_sk|c_salutation|c_first_name|c_last_name|c_preferred_cust_flag|c_birth_day|c_birth_month|c_birth_year|   c_birth_country|c_login|     c_email_address|c_last_review_date_sk|
+-------------+----------------+------------------+------------------+-----------------+----------------------+---------------------+------------+------------+-----------+---------------------+-----------+-------------+------------+------------------+-------+--------------------+---------------------+
|            2|AAAAAAAACAAAAAAA|            819667|              1461|          3681655|   

In [19]:
df_customer_100B.writeTo(f"{catalog_name}.{database_name}.{table_name}").append()

In [ ]:
spark.table(f"{catalog_name}.{database_name}.{table_name}.history") \
    .show()
     

In [22]:
update_Df = df_customer.filter(df_customer.c_birth_country=='UNITED STATES')
update_Df = update_Df.withColumn("c_birth_country",lit("US"))
update_Df.select(['c_customer_id']).distinct().count()

9206


In [48]:
update_Df.writeTo(f"{catalog_name}.{database_name}.{table_name}").append()

In [46]:
spark.table(f"{catalog_name}.{database_name}.{table_name}.history") \
    .show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-09-19 14:38:...|1437657406398650969|               null|               true|
|2024-09-19 14:41:...|5098578219220757586|1437657406398650969|               true|
|2024-09-19 14:43:...|7724982400463503082|5098578219220757586|               true|
|2024-09-19 14:56:...|4149979908348128478|7724982400463503082|               true|
|2024-09-19 15:03:...|2039670241441472078|4149979908348128478|               true|
+--------------------+-------------------+-------------------+-------------------+


In [40]:
### Run optimize or vaccum operation from athena ####